In [24]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [25]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("API_KEY not set")
else:
    print("API_KEY set")

API_KEY set


In [21]:
openai=OpenAI()

In [26]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrevalent in soup.body(["script", "style","img","input"]):
            irrevalent.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [29]:
ed=Website("https://www.google.com")
print(ed.title)
print(ed.text)

Google
Google
Search
Images
Maps
Play
YouTube
News
Gmail
Drive
More
»
Web History
|
Settings
|
Sign in
Advanced search
Google offered in:
हिन्दी
বাংলা
తెలుగు
मराठी
தமிழ்
ગુજરાતી
ಕನ್ನಡ
മലയാളം
ਪੰਜਾਬੀ
Advertising
Business Solutions
About Google
Google.co.in
© 2025 -
Privacy
-
Terms


In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [38]:
print(user_prompt_for(ed))

You are looking at a website titled Google
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Google
Search
Images
Maps
Play
YouTube
News
Gmail
Drive
More
»
Web History
|
Settings
|
Sign in
Advanced search
Google offered in:
हिन्दी
বাংলা
తెలుగు
मराठी
தமிழ்
ગુજરાતી
ಕನ್ನಡ
മലയാളം
ਪੰਜਾਬੀ
Advertising
Business Solutions
About Google
Google.co.in
© 2025 -
Privacy
-
Terms


In [40]:
def message_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)},
    ]

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=message_for(website),
    )
    return response.choices[0].message.content


#summarize("https://www.google.com")
message_for(ed)


[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Google\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nGoogle\nSearch\nImages\nMaps\nPlay\nYouTube\nNews\nGmail\nDrive\nMore\n»\nWeb History\n|\nSettings\n|\nSign in\nAdvanced search\nGoogle offered in:\nहिन्दी\nবাংলা\nతెలుగు\nमराठी\nதமிழ்\nગુજરાતી\nಕನ್ನಡ\nമലയാളം\nਪੰਜਾਬੀ\nAdvertising\nBusiness Solutions\nAbout Google\nGoogle.co.in\n© 2025 -\nPrivacy\n-\nTerms'}]